In [1]:
import os
import sys

import numpy as np
#sys.path.insert(1, r"C:\Users\Zack\OneDrive\BCI\arl-eegmodels-master")
#from EEGModels import ShallowConvNet

# main directory
main_dir = os.getcwd()
# eeg data directory
eeg_dir = os.path.join(main_dir, "eeg_data")
# eeg validation data directory
eeg_validation_dir = os.path.join(main_dir, "eeg_data_validation")

MI_CLASSES = ["left", "right", "none"]

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def to_one_sec(eeg_data, num_sec=5, samp_freq=250):
    one_sec_data = []
    for i in range(num_sec):
        eeg_data_i = eeg_data[i*samp_freq:(i+1)*samp_freq]
        one_sec_data.append(eeg_data_i)

    return one_sec_data

In [22]:
def create_data(data_dir):
    training_data_dict = {}

    for mi_class in MI_CLASSES:
        if mi_class not in training_data_dict:
            training_data_dict[mi_class] = []

        mi_class_dir = os.path.join(data_dir, mi_class)
        for filename in os.listdir(mi_class_dir):
            if "filtered" in filename:
                eeg_data = np.loadtxt(os.path.join(mi_class_dir, filename), delimiter=',')
                one_sec_list = to_one_sec(eeg_data, num_sec=5, samp_freq=250)
                for i in range(len(one_sec_list)):  # for each 1 second trial
                    one_sec_data = one_sec_list[i] # (250, 8) eeg data
                    one_sec_dataT = np.transpose(one_sec_data)  # (8, 250) eeg data
                    training_data_dict[mi_class].append(one_sec_dataT)  

    session_count = [len(training_data_dict[mi_class]) for mi_class in MI_CLASSES]
    print(f"One sec trial count:\nLeft: {session_count[0]}, Right: {session_count[1]}, None: {session_count[2]}\n")

    for mi_class in MI_CLASSES:
        np.random.shuffle(training_data_dict[mi_class])  # randomize session order
        training_data_dict[mi_class] = training_data_dict[mi_class][:min(session_count)]  # use min session count number of sessions

    # creating X, y 
    labeled_data = []
    for mi_class in MI_CLASSES:
        for data in training_data_dict[mi_class]:
            if mi_class == "left":
                labeled_data.append([data, [1, 0, 0]])
            elif mi_class == "right":
                labeled_data.append([data, [0, 1, 0]])
            elif mi_class == "none":
                labeled_data.append([data, [0, 0, 1]])

    np.random.shuffle(labeled_data)
    
    return labeled_data


In [23]:
labeled_data = create_data(data_dir=eeg_dir)

One sec trial count:
Left: 20, Right: 20, None: 20



In [25]:
print("Creating training data...")
train_data = create_data(data_dir=eeg_dir)
train_X = []
train_y = []

for X, y in train_data:
    train_X.append(X)
    train_y.append(y)

print("Creating testing data...")
test_data = create_data(data_dir=eeg_validation_dir)
test_X = []
test_y = []

for X, y in test_data:
    test_X.append(X)
    test_y.append(y)

Creating training data...
One sec trial count:
Left: 20, Right: 20, None: 20

Creating testing data...
One sec trial count:
Left: 5, Right: 5, None: 5



In [27]:
#train_X = np.transpose(np.array(train_X))
train_X = np.array(train_X)
print("train_X shape: " + str(np.shape(train_X)))
#test_X = np.transpose(np.array(test_X))
test_X = np.array(test_X)
print("test_X shape: " + str(np.shape(test_X)))

#train_y = np.transpose(np.array(train_y))
train_y = np.array(train_y)
print("train_y shape: " + str(np.shape(train_y)))
#test_y = np.transpose(np.array(test_y))
test_y = np.array(test_y)
print("test_y shape: " + str(np.shape(test_y)))

train_X shape: (60, 8, 250)
test_X shape: (15, 8, 250)
train_y shape: (60, 3)
test_y shape: (15, 3)


In [71]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, AveragePooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [51]:
    def square(x):
        return K.square(x)

    def log(x):
        return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

    def ShallowConvNet(nb_classes, Chans = 8, Samples = 250, dropoutRate = 0.5):
        # start the model
        input_main   = Input((Chans, Samples, 1))
        block1       = Conv2D(40, (1, 25), 
                                        input_shape=(Chans, Samples, 1),
                                        kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
        block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                                kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
        block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
        block1       = Activation(square)(block1)
        block1       = AveragePooling2D(pool_size=(1, 75), strides=(1, 15))(block1)
        block1       = Activation(log)(block1)
        block1       = Dropout(dropoutRate)(block1)
        flatten      = Flatten()(block1)
        dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
        softmax      = Activation('softmax')(dense)

        return Model(inputs=input_main, outputs=softmax)

In [64]:
model = ShallowConvNet(nb_classes=3, Chans=8, Samples=250)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 8, 250, 1)]       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 226, 40)        1040      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 1, 226, 40)        12800     
_________________________________________________________________
batch_normalization_7 (Batch (None, 1, 226, 40)        160       
_________________________________________________________________
activation_21 (Activation)   (None, 1, 226, 40)        0         
_________________________________________________________________
average_pooling2d_7 (Average (None, 1, 11, 40)         0         
_________________________________________________________________
activation_22 (Activation)   (None, 1, 11, 40)         0   

In [65]:
model.compile(loss="categorical_crossentropy", optimizer ="adam", metrics=["accuracy"])

In [67]:
epochs = 50
batch_size = 10
#fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=50, validation_data=(test_X, test_y))
fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, verbose=2)


Epoch 1/50
6/6 - 1s - loss: 0.9931 - accuracy: 0.7000
Epoch 2/50
6/6 - 0s - loss: 0.4286 - accuracy: 0.8333
Epoch 3/50
6/6 - 0s - loss: 1.4098 - accuracy: 0.5000
Epoch 4/50
6/6 - 0s - loss: 0.4282 - accuracy: 0.8833
Epoch 5/50
6/6 - 0s - loss: 0.3046 - accuracy: 0.9167
Epoch 6/50
6/6 - 0s - loss: 0.3463 - accuracy: 0.8333
Epoch 7/50
6/6 - 0s - loss: 0.2480 - accuracy: 0.9500
Epoch 8/50
6/6 - 0s - loss: 0.4689 - accuracy: 0.8333
Epoch 9/50
6/6 - 0s - loss: 0.2019 - accuracy: 0.9333
Epoch 10/50
6/6 - 0s - loss: 0.7164 - accuracy: 0.6167
Epoch 11/50
6/6 - 0s - loss: 0.3186 - accuracy: 0.9000
Epoch 12/50
6/6 - 0s - loss: 0.4124 - accuracy: 0.8833
Epoch 13/50
6/6 - 0s - loss: 0.3783 - accuracy: 0.8667
Epoch 14/50
6/6 - 0s - loss: 0.2314 - accuracy: 0.9500
Epoch 15/50
6/6 - 0s - loss: 0.1853 - accuracy: 0.9500
Epoch 16/50
6/6 - 0s - loss: 0.4508 - accuracy: 0.8000
Epoch 17/50
6/6 - 0s - loss: 0.2381 - accuracy: 0.9000
Epoch 18/50
6/6 - 0s - loss: 0.2031 - accuracy: 0.9167
Epoch 19/50
6/6 - 0

In [75]:
model = Sequential()

model.add(Conv1D(64, (3), input_shape=np.shape(train_X)))
model.add(Activation('relu'))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())

model.add(Dense(512))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

ValueError: Input 0 of layer max_pooling1d_2 is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 60, 5, 64)